In [1]:
# An example for saving species into an RMG thermo library

In [31]:
import os
import rmgpy.data.kinetics
import rmgpy.kinetics
import numpy as np

# Construct some example species with thermo

In [32]:
my_rxn1 = rmgpy.reaction.Reaction()
my_rxn1.reactants = [
    rmgpy.species.Species(smiles='CCCC'),
    rmgpy.species.Species(smiles='[OH]'),
]
my_rxn1.products = [
    rmgpy.species.Species(smiles='[CH2]CCC'),
    rmgpy.species.Species(smiles='O'),
]
display(my_rxn1)
my_rxn1.kinetics = rmgpy.kinetics.Arrhenius(A=(3.31e1, 'm^3/(mol*s)'), n=1.8, Ea=(3990, 'J/mol'))


my_rxn2 = rmgpy.reaction.Reaction()
my_rxn2.reactants = [
    rmgpy.species.Species(smiles='[CH2]CCC'),
    rmgpy.species.Species(smiles='[O][O]'),
]
my_rxn2.products = [
    rmgpy.species.Species(smiles='C=CCC'),
    rmgpy.species.Species(smiles='[O]O'),
]
display(my_rxn2)
my_rxn2.kinetics = rmgpy.kinetics.Arrhenius(A=(2.7e5, 'm^3/(mol*s)'), n=0, Ea=(0, 'J/mol'))

my_reactions = [my_rxn1, my_rxn2]

# Write the species kinetics to a library

In [33]:
rmg_kinetics_db = rmgpy.data.kinetics.KineticsDatabase()
rmg_kinetics_db.libraries['cool_kinetics'] = rmgpy.data.kinetics.KineticsLibrary() # <-------- this determines the name of the directory holding reactions.py and dictionary.txt
rmg_kinetics_db.libraries['cool_kinetics'].name = 'my_kinetics_title'  # <------- this is the name of the library inside the file
for rxn in my_reactions:
    entry = rmgpy.data.base.Entry()
    entry.label = str(rxn)
    entry.item = rxn
    entry.data = rxn.kinetics
    rmg_kinetics_db.libraries['cool_kinetics'].entries[entry.label] = entry

# save the results
rmg_kinetics_db.save_libraries('kinetics')  # <----------- this will be the name of the directory containing the cool_kinetics folder

In [34]:
rmg_kinetics_db.libraries['cool_kinetics'].entries

OrderedDict([('CCCC + [OH] <=> [CH2]CCC + O',
              <Entry index=0 label="CCCC + [OH] <=> [CH2]CCC + O">),
             ('[CH2]CCC + [O][O] <=> C=CCC + [O]O',
              <Entry index=1 label="[CH2]CCC + [O][O] <=> C=CCC + [O]O">)])

# Read the kinetics library back in

In [35]:
# try reading to test
# Load the new thermo library

kinetics_lib = os.path.join('kinetics')
rmg_kinetics_db2 = rmgpy.data.kinetics.KineticsDatabase()
rmg_kinetics_db2.load_libraries(kinetics_lib)
print(f'{len(rmg_kinetics_db2.libraries["cool_kinetics"].entries)} entries loaded')

# print the enthalpy at 1000 K just for the heck of it
for key in rmg_kinetics_db2.libraries["cool_kinetics"].entries.keys():
    kinetics = rmg_kinetics_db2.libraries["cool_kinetics"].entries[key].data
    name = rmg_kinetics_db2.libraries["cool_kinetics"].entries[key].label
    print(f'{name}\tlog10(k(T=1000):\t{np.log10(kinetics.get_rate_coefficient(1000)):0.2f} {kinetics.A.units}')

2 entries loaded
CCCC + [OH] <=> [CH2]CCC + O	log10(k(T=1000):	6.71 m^3/(mol*s)
[CH2]CCC + [O][O] <=> C=CCC + [O]O	log10(k(T=1000):	5.43 m^3/(mol*s)


In [36]:
rmg_kinetics_db2.libraries['cool_kinetics'].name

'my_kinetics_title'